# **1) Load tensorflow and check GPU**
Force Google Colab to use Tensorflow 1.X and check if tensorflow can connect to Colab's GPU.
If no GPU can be found, please check *Edit* -> *Settings* and select "GPU" as hardware accelerator.

In [ ]:
%tensorflow_version 1.x
!pip install tensorflow-gpu==1.15
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# **2) Install Tensorflow Object Detection API**
The main purpose of this snippet is to clone the [Tensorflow Models](https://github.com/tensorflow/models.git) repository. Additionally, a few additional packages are required and installed by pip. In the last step, the cloned packages are added to the python path.


In [ ]:
!pip install tf_slim
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib
!pip install -q pycocotools
%cd /content/models/research

#Fixes a common bug in Tensorflow Object Detection API
!protoc object_detection/protos/*.proto --python_out=.

#Add Tensorflow model to python environment
import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

# **3) Select pretrained model**
For easier evaluation of different models, different pretrained models from the [Object Detection API](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf1_detection_zoo.md) have been preconfigured.

- *model_name*: file name of the model tar.gzip as given in the Tensorflow Model Zoo
- *pipeline_file*: According pipeline file
- *batch_size*: Highest batch size that doesn't exceed GPU memory

In [ ]:
# Number of training steps.
num_steps = 6000  # 200000

# Number of evaluation steps.
num_eval_steps = 50

MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
        'batch_size': 12
    },
    'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
        'pipeline_file': 'faster_rcnn_inception_v2_pets.config',
        'batch_size': 12
    },
    'rfcn_resnet101': {
        'model_name': 'rfcn_resnet101_coco_2018_01_28',
        'pipeline_file': 'rfcn_resnet101_pets.config',
        'batch_size': 8
    },
    'faster_rcnn_resnet50': {
        'model_name': 'faster_rcnn_resnet50_coco_2018_01_28',
        'pipeline_file': 'faster_rcnn_resnet50_coco.config',
        'batch_size': 12
    }
}

# Pick the model you want to use
selected_model = 'faster_rcnn_resnet50'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size fits in Colabe's Tesla K80 GPU memory for selected model.
batch_size = MODELS_CONFIG[selected_model]['batch_size']

# **4) Download pretrained model**
Download the selected pretrained model from tensorflow and move it to the right place.

In [ ]:
%cd /content/models/research

import os
import shutil
import glob
import urllib.request
import tarfile
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/content/models/research/pretrained_model'

#Check if models has already been downloaded
if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

#Unzip
tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

!echo {DEST_DIR}
!ls -alh {DEST_DIR}

In [ ]:
fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
fine_tune_checkpoint

'/content/models/research/pretrained_model/model.ckpt'

## **5) Load training data**
The model requires three files:

*   *train.record:* TFRecord file holding all examples for training.
*   *test.record:* TFRecord file holding examples for testing.
*   *label_map.pbtxt:* JSON file mapping class names to class IDs.

Since these files are pretty large and Google Colab deletes custom data after disconnection, these files are stored on Google Drive that gets mounted into the Colab environment. Therefore it is required to provide this Colab notebook a one-time key to access the Drive. Please follow the instructions given when executing this cell.





In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Set path variables to test and train data as well as labels.
train_record_fname = '/content/drive/My Drive/TFRecord/Max_800px/train.record'
test_record_fname = '/content/drive/My Drive/TFRecord/Max_800px/test.record'
label_map_pbtxt_fname = '/content/drive/My Drive/TFRecord/Max_800px/label_map.pbtxt'

## **6) Configuring Training Pipeline**

In [ ]:
import os

# Load model's pipeline file
pipeline_fname = os.path.join('/content/models/research/object_detection/samples/configs/', pipeline_file)

assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)

Each pretrained Tensorflow model comes with a so called [pipeline file](https://github.com/tensorflow/models/blob/master/research/object_detection/protos/pipeline.proto) that contains the most important settings. We have to adjust this, e.g. with our batch size, input data, etc.

We add the following information:
*   Path to fine_tune_checkpoint (file where model parameters are stored)
*   Path to train and test data (as configured above)
*   Path to label map (as configured above)
*   Batch_size
*   Training_steps
*   Evaluation_steps
*   Number of classes (right now just stairs = 1 class)



In [ ]:
import re

with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    
    num_classes = 1

    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    f.write(s)

In [ ]:
# Print adjusted pipeline
!cat {pipeline_fname}

model_dir = 'training/'

# Remove content in output model directory to fresh start.
!rm -rf {model_dir}
os.makedirs(model_dir, exist_ok=True)

# **7) Initialize Tensorboard**
[Tensorboard](https://www.tensorflow.org/tensorboard) is a great tool to visualize the training model, important metrics and examplatory results.

Unfortunately, we cannot access it directly through Google Colab and have to use a tunnel. [Ngrok](https://ngrok.com/product) offers such a tunnel.


In [ ]:
# Download and unzip Ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

# Initialize tensorboard and expose it to port 6000
LOG_DIR = model_dir
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [ ]:
# Expose port 6000 (used by tensorboard) via ngrok tunnel to public
get_ipython().system_raw('./ngrok http 6006 &')

In [ ]:
# Print public address to exposed tensorboard
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://0294338ce352.ngrok.io


# **8) Train model**
This cell finally trains the model with the provided training data. Real time results can be access via Tensorboard.

In [ ]:
!python /content/models/research/object_detection/model_main.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --num_eval_steps={num_eval_steps}

## **9) Export trained model**
There are to files to be exported:


1.   The inference graph
2.   The model.pb file



In [ ]:
import re
import numpy as np

# Export to new folder "fine_tuned_model"
output_directory = './fine_tuned_model'

# Find last tensorflow checkpoint file in model directory
lst = os.listdir(model_dir)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)

# export graph by using a Tensorflow API script
!python /content/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={pipeline_fname} \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}

In [ ]:
# Check if we really exported something
!ls {output_directory}

In [ ]:
import os

pb_fname = os.path.join(os.path.abspath(output_directory), "frozen_inference_graph.pb")
assert os.path.isfile(pb_fname), '`{}` not exist'.format(pb_fname)

In [ ]:
# Check if we really exported something
!ls -alh {pb_fname}

## **10) Download model files**
Otherwise, one can also copy the folder *fine_tuned_model* to the Google Drive. Warning: After closing the Colab (or alternatively some inactive time) all data in this Colab gets lost!

In [ ]:
!zip -r /content/file.zip /content/models/research/fine_tuned_model/
from google.colab import files
files.download("/content/file.zip")
